In [1]:
import matplotlib.pyplot as plt
from uniswap_fee_and_divergence import *

In [2]:
dataset = pd.read_csv("Backtest_ready_ETHUSDT_1H.csv", index_col=0, parse_dates=True)

# Fee multiplier estimation
def estimate_conc_liq_fee_multipler(range_pct: float) -> float:
    return USD_FEE_PER_VOLUME * (6 / range_pct)

INITIAL_CAPITAL_USD = 100_000
EXCHANGE_FEE = 0.0002
IS_HEDGED = [True, False]
RANGE_PCTS = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0.75, 0.5, 0.25]
# 2.01$ from $100 principal over last 10 days (May 28th - June 7th)
# Get last 10 day's cumulative volume
VOLUME_SUM = dataset["volume"]["2023-05-28":"2023-06-08"].sum() # type: ignore
USD_FEE_CAPITAL_MULTIPLIER = INITIAL_CAPITAL_USD/100
USD_FEE_PER_VOLUME = 2.01 / VOLUME_SUM
USD_FEE_PER_VOLUME = USD_FEE_PER_VOLUME * USD_FEE_CAPITAL_MULTIPLIER


In [3]:
import pathos.multiprocessing as mp
mp.cpu_count()
from pathos.multiprocessing import ProcessingPool as Pool
from tqdm import tqdm
import itertools


# function to run backtest
def run_backtest(params):
    range_pct, is_hedged = params
    backtester = DataBacktester(
        data=dataset, 
        capital_usd=INITIAL_CAPITAL_USD, 
        range_pct=range_pct, 
        fee_per_volume=estimate_conc_liq_fee_multipler(range_pct),
        exchange_fee=EXCHANGE_FEE,
        is_hedged=is_hedged)
    
    # Run backtest
    return {
        "range_pct": range_pct,
        "is_hedged": is_hedged,
        "backtester": backtester.run(),
    }

# generate all combinations of range_pct and hedge_pct
params = list(itertools.product(RANGE_PCTS, IS_HEDGED))

# create a multiprocessing Pool with cpu-1 processes
cpu_count = int(mp.cpu_count() / 2)
pool = Pool(processes=cpu_count)

# run the backtests in parallel with a progress bar
results = []
for result in tqdm(pool.imap(run_backtest, params), total=len(params)):
    results.append(result)

# close the pool to free up system resources
pool.close()
pool.join()


100%|██████████| 26/26 [00:38<00:00,  1.49s/it]


In [4]:
# Save pickle
import pickle
with open("backtest_results.pickle", "wb") as f:
    pickle.dump(results, f)